In [10]:
cache_dir = "./fireball_data_cache/"

import re
from os import listdir
from collections import Counter, defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta

event_date_pattern = re.compile("evcorr_\d{8}_(\d{8})_\d{6}[A-Z]_orbit.txt")


events_by_year = defaultdict(Counter)

for file in listdir(cache_dir):
    match = event_date_pattern.match(file)

    if not match:
        continue

    event_date = datetime.strptime(match.groups()[0],"%Y%m%d")
    
    #events_by_year[event_date.year][event_date]+=1
    
    events_by_year[event_date.year][event_date] += 1
    
    

In [11]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show, output_file, vplot
output_notebook()
#p = figure(x_axis_type = "datetime")
p = figure(title="Fireballs Reported by NASA ASGARD", plot_width=900)


def kludge_year(kludge):
    def _inner(d):
        return d.replace(year=kludge)
    return _inner


def days_since_jan1(d):
    return (d - d.replace(month=1, day=1)).days



colors = [
    "blue",
    "green",
    "red",
    "black",
    "darkcyan",
    "goldenrod"
]

from math import log

color_index = 0
for year in events_by_year.keys():
    x_values, y_values = zip(*sorted(events_by_year[year].items(), key=lambda item: item[0]))    
    x_ticks = list(map(days_since_jan1, x_values))
    
    p.line(x_ticks, y_values, color=colors[color_index])#, legend=str(year))
    color_index+=1

#x2011, y2011= zip(*sorted(events_by_year[2011].items(), key=lambda item: item[0]))
# x2012, y2012= zip(*sorted(events_by_year[2012].items(), key=lambda item: item[0]))
# x2013, y2013= zip(*sorted(events_by_year[2013].items(), key=lambda item: item[0]))




#x2011_ticks = list(map(days_since_jan1, x2011))
# x2012_ticks = list(map(days_since_jan1, x2012))
# x2013_ticks = list(map(days_since_jan1, x2013))

# # print(x2011_ticks)

#p.line(x2011_ticks, y2011, color="red", legend='2011')
# p.line(x2012_ticks, y2012, color="blue", legend='2012')
# p.line(x2013_ticks, y2013, color="green", legend='2013')

# from bokeh.models.formatters import DatetimeTickFormatter
#p.xaxis.formatter = DatetimeTickFormatter(formats=dict(days=["%a\n%d %b"]))
#p = figure()
#p.circle(5, 5, radius=50)


from bokeh.models import Range1d
p.y_range = Range1d(-14, 100)
p.legend.location = "top_center"
# print(dir(p.legend))


p.xaxis.axis_label = 'Day of the Year'
p.yaxis.axis_label = 'Total Fireballs Detected'

shower_label_y_offset = -4

p.text(3, shower_label_y_offset, text=["Quadrantid"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(18, shower_label_y_offset*2, text=["Jan Xi Ursae Majorids"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(99, shower_label_y_offset*3, text=["Zeta Cygnids"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(110, shower_label_y_offset, text=["Lyrid"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(125, shower_label_y_offset * 2, text=["Eta Aquarid"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(225, shower_label_y_offset, text=["Perseid"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(294, shower_label_y_offset, text=["Orionid"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(322, shower_label_y_offset, text=["Leonid"], text_color="firebrick", text_align="center", text_font_size="10pt")
p.text(348, shower_label_y_offset, text=["Geminid"], text_color="firebrick", text_align="center", text_font_size="10pt")

show(p)


Loading BokehJS ...

In [12]:
totals = Counter()
averages = dict()

for year, events_for_year in events_by_year.items():
    for day, events_for_day in events_for_year.items():
        totals[days_since_jan1(day)] += events_for_day
        
number_of_years = len(events_by_year)
for day, count in totals.items():
    averages[day] = count / number_of_years

p2 = figure(title="Fireballs / Day", plot_width=900)

p2.y_range = Range1d(0, 8)

#print(totals)

x_values, y_values = zip(*sorted(totals.items(), key=lambda item: item[0])) 

p2.line(x_values, list(map(log, y_values)), color="black")

# x_values, y_values = zip(*sorted(averages.items(), key=lambda item: item[0])) 
# p2.line(x_values, y_values, color="red")




p2.xaxis.axis_label = 'Day of the Year'
p2.yaxis.axis_label = 'Total Fireballs Detected (log)'

show(p2)

In [13]:

averages = dict()

number_of_years = len(events_by_year)
for day, count in totals.items():
    averages[day] = count / number_of_years

p3 = figure(title="Fireballs / Day", plot_width=900)

p3.y_range = Range1d(0, 100)

x_values, y_values = zip(*sorted(totals.items(), key=lambda item: item[0])) 

# from numpy.fft import fft

# y_values = fft(y_values).tolist()



p3.line(x_values, y_values, color="red")




p3.xaxis.axis_label = 'Day of the Year'
p3.yaxis.axis_label = 'Avg Fireballs Detected'

show(p3)

In [14]:
import statistics
fireball_stddev = statistics.stdev(averages.values())
fireball_mean = statistics.mean(averages.values())

print(fireball_stddev, fireball_mean)


      
      
      


12.322722741436822 8.625227686703097


In [15]:
sum = 0
from statistics import mean, stdev

all_data = [val for years in events_by_year.values() for val in years.values()]

daily_mean = mean(all_data)
daily_stddev = stdev(all_data)

print(daily_mean, daily_stddev)

# define what would be normal...

filtered_data = list(filter(lambda i: i > daily_mean, all_data))
#print(filtered_data)

filtered_mean = mean(filtered_data)
filtered_stddev = stdev(filtered_data)
print(filtered_mean, filtered_stddev)


# for year, events_for_year in events_by_year.items():
#     for day, events_for_day in events_for_year.items():
#         totals[days_since_jan1(day)] += events_for_day
        

11.852941176470589 20.433321485033993
26.230174081237912 31.097110003198566


In [16]:
# for i, val in enumerate(c):
# mean(c[max(i-15, 0):min(i+15, len(c))]))

grouping_range = 3

grouped_averages = dict()

for year in events_by_year.keys():
    x_values, y_values = zip(*sorted(events_by_year[year].items(), key=lambda item: item[0]))    
#     x_values = list(map(days_since_jan1, x_values))
    
    
    new_data = []
    for i, val in enumerate(y_values):
        start = max(i-grouping_range, 0)
        stop = min(i+grouping_range+1, len(y_values))
        
        new_data.append(mean(y_values[start:stop]))
    
    grouped_averages[year] = dict(zip(x_values, new_data))
    

In [17]:
p4 = figure(title="2012 # of Fireballs to 7 day average", plot_width=900)

p4.y_range = Range1d(0, 100)

x_values, y_values = zip(*sorted(events_by_year[2012].items(), key=lambda item: item[0]))    
x_ticks = list(map(days_since_jan1, x_values))

avg_x_values, avg_y_values = zip(*sorted(grouped_averages[2012].items(), key=lambda item: item[0]))    

p4.line(x_ticks, y_values, color="red", legend="Data")
p4.line(x_ticks, avg_y_values, color="black", legend="Avg")

p4.xaxis.axis_label = 'Day of the Year'
p4.yaxis.axis_label = 'Fireballs Detected'

show(p4)